# Hate Speech Detection Project

In [22]:
import sagemaker
import pandas as pd
import numpy as np

In [24]:
tweets_amlo = pd.read_csv("data/tweets_amlo.txt", sep=";")

In [169]:
tweets_amlo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13586 entries, 0 to 13585
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user             13586 non-null  object 
 1   date_time        13586 non-null  object 
 2   likes            13586 non-null  int64  
 3   tweet            13586 non-null  object 
 4   tokenized_tweet  13586 non-null  object 
 5   offensive        13586 non-null  int64  
 6   sentiment        13586 non-null  float64
 7   hate_speech      13586 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 849.2+ KB


In [106]:
tweets_amlo["hate_speech"].value_counts()

0    11477
1     2109
Name: hate_speech, dtype: int64

In [108]:
data_input = tweets_amlo[['offensive', 'sentiment', 'hate_speech']]

In [134]:
x = data_input[['offensive', 'sentiment']]
y = data_input.hate_speech

In [135]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [136]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(10868, 2) (2718, 2) (10868,) (2718,)


In [137]:
def make_csv(x, y, filename, data_dir):
    '''Merges features and labels and converts them into one csv file with labels in the first column.
       :param x: Data features
       :param y: Data labels
       :param file_name: Name of csv file, ex. 'train.csv'
       :param data_dir: The directory where files will be saved
       '''

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    x_df = pd.DataFrame(x)
    y_df = pd.Series(y)
    csv_file = pd.concat([y_df, x_df], axis = 1)
    csv_file = csv_file.dropna()
    csv_file.to_csv(os.path.join(data_dir, filename), header = False, index = False)
    
    print('Path created: '+str(data_dir)+'/'+str(filename))

In [138]:
make_csv(x_train, y_train, filename='train.csv', data_dir='hate_speech_data')
make_csv(x_test, y_test, filename='test.csv', data_dir='hate_speech_data')

Path created: hate_speech_data/train.csv
Path created: hate_speech_data/test.csv


In [139]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [140]:
data_dir = 'hate_speech_data'

prefix = 'hate_speech_detection'

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print(input_data)

s3://sagemaker-us-east-1-991156428171/hate_speech_detection


In [145]:
from sagemaker.pytorch import PyTorch

output_path = 's3//{}/{}'.format(bucket, prefix)

estimator = PyTorch(entry_point = 'train.py',
                    source_dir = 'source',
                    role = role,
                    framework_version = '1.0',
                    train_instance_count = 1,
                    train_instance_type = 'ml.c4.xlarge',
                    sagemaker_session = sagemaker_session,
                    hyperparameters = {
                        'input_features':2,
                        'hidden_dim':20,
                        'output_dim':1,
                        'epochs':3000
                    })

In [146]:
%%time

estimator.fit({'train':input_data})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2021-02-28 09:22:12 Starting - Starting the training job...
2021-02-28 09:22:15 Starting - Launching requested ML instances.........
2021-02-28 09:23:47 Starting - Preparing the instances for training...
2021-02-28 09:24:39 Downloading - Downloading input data...
2021-02-28 09:25:07 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-02-28 09:25:08,678 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-02-28 09:25:08,681 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-02-28 09:25:08,693 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-02-28 09:25:09,326 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-02-28 09:25:09,597 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setu

Epoch: 41, Loss: 0.049686706912648426
Epoch: 42, Loss: 0.0485274362559226
Epoch: 43, Loss: 0.04742681197793328
Epoch: 44, Loss: 0.046380797492677296
Epoch: 45, Loss: 0.045385627330725944
Epoch: 46, Loss: 0.044437634431649005
Epoch: 47, Loss: 0.04353361933615074
Epoch: 48, Loss: 0.04267072431227638
Epoch: 49, Loss: 0.04184624478723956
Epoch: 50, Loss: 0.041058589689447825
Epoch: 51, Loss: 0.04030512847322284
Epoch: 52, Loss: 0.03958365578893731
Epoch: 53, Loss: 0.0388920453943691
Epoch: 54, Loss: 0.03822835812087266
Epoch: 55, Loss: 0.03759107235863259
Epoch: 56, Loss: 0.036978564790366986
Epoch: 57, Loss: 0.03638952578833401
Epoch: 58, Loss: 0.035822736811218524
Epoch: 59, Loss: 0.035277007301536635
Epoch: 60, Loss: 0.034751270646562464
Epoch: 61, Loss: 0.03424430784060974
Epoch: 62, Loss: 0.03375505963289605
Epoch: 63, Loss: 0.03328264462704118
Epoch: 64, Loss: 0.032826168174622604
Epoch: 65, Loss: 0.032384753739318636
Epoch: 66, Loss: 0.03195770034290913
Epoch: 67, Loss: 0.0315442880

Epoch: 220, Loss: 0.012777324063632273
Epoch: 221, Loss: 0.012737183685175376
Epoch: 222, Loss: 0.01269736726469913
Epoch: 223, Loss: 0.012657859348213495
Epoch: 224, Loss: 0.012618664847206348
Epoch: 225, Loss: 0.012579779259002818
Epoch: 226, Loss: 0.012541204950711132
Epoch: 227, Loss: 0.012502935289704939
Epoch: 228, Loss: 0.012464957629978396
Epoch: 229, Loss: 0.012427278822205267
Epoch: 230, Loss: 0.012389895723262604
Epoch: 231, Loss: 0.012352800938601409
Epoch: 232, Loss: 0.012315983342347929
Epoch: 233, Loss: 0.012279452323897246
Epoch: 234, Loss: 0.012243195979204665
Epoch: 235, Loss: 0.01220721099748621
Epoch: 236, Loss: 0.012171491902217248
Epoch: 237, Loss: 0.012136044744338699
Epoch: 238, Loss: 0.012100868037233274
Epoch: 239, Loss: 0.012065946588356182
Epoch: 240, Loss: 0.012031272361943159
Epoch: 241, Loss: 0.011996857077920586
Epoch: 242, Loss: 0.0119626894907915
Epoch: 243, Loss: 0.011928771519100722
Epoch: 244, Loss: 0.011895100387207756
Epoch: 245, Loss: 0.011861663

Epoch: 407, Loss: 0.008408641083437493
Epoch: 408, Loss: 0.00839484025604152
Epoch: 409, Loss: 0.008381092018828555
Epoch: 410, Loss: 0.00836740103477725
Epoch: 411, Loss: 0.00835376452131767
Epoch: 412, Loss: 0.008340185431997165
Epoch: 413, Loss: 0.00832665993407641
Epoch: 414, Loss: 0.008313193451410454
Epoch: 415, Loss: 0.008299783526985646
Epoch: 416, Loss: 0.008286426254189431
Epoch: 417, Loss: 0.008273119729129669
Epoch: 418, Loss: 0.008259866752063938
Epoch: 419, Loss: 0.008246669838611357
Epoch: 420, Loss: 0.008233528309516258
Epoch: 421, Loss: 0.008220439517219513
Epoch: 422, Loss: 0.008207399860650418
Epoch: 423, Loss: 0.008194417755807634
Epoch: 424, Loss: 0.008181491448906406
Epoch: 425, Loss: 0.008168618061855764
Epoch: 426, Loss: 0.008155788698848074
Epoch: 427, Loss: 0.008143013710401487
Epoch: 428, Loss: 0.00813028900712515
Epoch: 429, Loss: 0.008117611352803759
Epoch: 430, Loss: 0.008104986629743289
Epoch: 431, Loss: 0.008092410490721466
Epoch: 432, Loss: 0.0080798847

...

Epoch: 2322, Loss: 0.002729103711619208
Epoch: 2323, Loss: 0.002728386249990224
Epoch: 2324, Loss: 0.002727666691815335
Epoch: 2325, Loss: 0.0027269450302011295
Epoch: 2326, Loss: 0.002726222237840686
Epoch: 2327, Loss: 0.002725504018434951
Epoch: 2328, Loss: 0.0027247842742535314
Epoch: 2329, Loss: 0.002724066044210619
Epoch: 2330, Loss: 0.00272334395341892
Epoch: 2331, Loss: 0.0027226275216639868
Epoch: 2332, Loss: 0.0027219074221080885
Epoch: 2333, Loss: 0.0027211923518186336
Epoch: 2334, Loss: 0.0027204769327743325
Epoch: 2335, Loss: 0.0027197608404958993
Epoch: 2336, Loss: 0.0027190448902289135
Epoch: 2337, Loss: 0.00271832707156668
Epoch: 2338, Loss: 0.002717613309282933
Epoch: 2339, Loss: 0.0027168936209116964
Epoch: 2340, Loss: 0.0027161782902296516
Epoch: 2341, Loss: 0.002715470152537061
Epoch: 2342, Loss: 0.002714757038841997
Epoch: 2343, Loss: 0.00271404137899533
Epoch: 2344, Loss: 0.002713331567920472
Epoch: 2345, Loss: 0.002712616029433576
Epoch: 2346, Loss: 0.002711907630

Epoch: 2494, Loss: 0.0026119776174441146
Epoch: 2495, Loss: 0.00261133770540407
Epoch: 2496, Loss: 0.0026106975285460675
Epoch: 2497, Loss: 0.002610061022236347
Epoch: 2498, Loss: 0.002609425242203642
Epoch: 2499, Loss: 0.0026087869484049627
Epoch: 2500, Loss: 0.0026081484356026974
Epoch: 2501, Loss: 0.002607512356446829
Epoch: 2502, Loss: 0.00260687450750519
Epoch: 2503, Loss: 0.0026062386138130493
Epoch: 2504, Loss: 0.002605604048380554
Epoch: 2505, Loss: 0.0026049689958193797
Epoch: 2506, Loss: 0.002604332988553068
Epoch: 2507, Loss: 0.002603698086835691
Epoch: 2508, Loss: 0.0026030615962143195
Epoch: 2509, Loss: 0.002602425343901089
Epoch: 2510, Loss: 0.0026017923165000518
Epoch: 2511, Loss: 0.0026011569781183162
Epoch: 2512, Loss: 0.0026005223207600505
Epoch: 2513, Loss: 0.0025998886630637747
Epoch: 2514, Loss: 0.0025992568298123806
Epoch: 2515, Loss: 0.002598624213764797
Epoch: 2516, Loss: 0.0025979923182913255
Epoch: 2517, Loss: 0.0025973611530845406
Epoch: 2518, Loss: 0.0025967

Epoch: 2664, Loss: 0.0025086723019901083
Epoch: 2665, Loss: 0.0025080921096981863
Epoch: 2666, Loss: 0.0025075114124175997
Epoch: 2667, Loss: 0.002506932184616328
Epoch: 2668, Loss: 0.002506349861686324
Epoch: 2669, Loss: 0.0025057678524296988
Epoch: 2670, Loss: 0.0025051891313331667
Epoch: 2671, Loss: 0.0025046112017812943
Epoch: 2672, Loss: 0.0025040309641738883
Epoch: 2673, Loss: 0.0025034583084897633
Epoch: 2674, Loss: 0.0025028788604710473
Epoch: 2675, Loss: 0.002502299191338093
Epoch: 2676, Loss: 0.0025017226508715403
Epoch: 2677, Loss: 0.0025011472592392334
Epoch: 2678, Loss: 0.0025005682046723007
Epoch: 2679, Loss: 0.0024999877141038728
Epoch: 2680, Loss: 0.0024994129520231156
Epoch: 2681, Loss: 0.002498838546302419
Epoch: 2682, Loss: 0.0024982633231644623
Epoch: 2683, Loss: 0.0024976910694917307
Epoch: 2684, Loss: 0.002497117760593739
Epoch: 2685, Loss: 0.0024965429108142743
Epoch: 2686, Loss: 0.0024959681010008577
Epoch: 2687, Loss: 0.002495397973825556
Epoch: 2688, Loss: 0.0

Epoch: 2836, Loss: 0.0024145446754404083
Epoch: 2837, Loss: 0.002414027601627179
Epoch: 2838, Loss: 0.0024135087811286956
Epoch: 2839, Loss: 0.0024129947187122415
Epoch: 2840, Loss: 0.0024124765880772425
Epoch: 2841, Loss: 0.0024119582142005467
Epoch: 2842, Loss: 0.0024114414360805335
Epoch: 2843, Loss: 0.002410927030270208
Epoch: 2844, Loss: 0.0024104079919856566
Epoch: 2845, Loss: 0.00240989357080354
Epoch: 2846, Loss: 0.002409377403077166
Epoch: 2847, Loss: 0.0024088624557365977
Epoch: 2848, Loss: 0.002408348120673228
Epoch: 2849, Loss: 0.002407831948647541
Epoch: 2850, Loss: 0.002407320391685084
Epoch: 2851, Loss: 0.0024068063190769043
Epoch: 2852, Loss: 0.0024062937908310832
Epoch: 2853, Loss: 0.0024057817367835305
Epoch: 2854, Loss: 0.0024052680934983028
Epoch: 2855, Loss: 0.0024047543682565366
Epoch: 2856, Loss: 0.002404242120954407
Epoch: 2857, Loss: 0.002403730882146193
Epoch: 2858, Loss: 0.002403219735595645
Epoch: 2859, Loss: 0.002402709114898522
Epoch: 2860, Loss: 0.0024021


2021-02-28 09:57:27 Uploading - Uploading generated training model
2021-02-28 09:57:27 Completed - Training job completed
Training seconds: 1968
Billable seconds: 1968
CPU times: user 5.44 s, sys: 279 ms, total: 5.72 s
Wall time: 35min 46s


In [147]:
%%time

from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(model_data = estimator.model_data,
                         role = role,
                         framework_version = '1.0',
                         entry_point = 'predict.py',
                         source_dir = 'source')


predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-----------------!CPU times: user 452 ms, sys: 47.3 ms, total: 499 ms
Wall time: 8min 33s


In [148]:
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

In [164]:
test_y_preds = np.squeeze(predictor.predict(test_x))

In [150]:
accuracy = np.mean(test_y_preds == test_y)

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

1.0

Predicted class labels: 
[0. 0. 0. ... 0. 0. 0.]

True class labels: 
[0 0 0 ... 0 0 0]


In [162]:
from sklearn.metrics import recall_score
recall_score(test_y.values, test_y_preds)

1.0

In [163]:
from sklearn.metrics import precision_score
precision_score(test_y.values, test_y_preds)

1.0

In [168]:
from sklearn.metrics import classification_report
print(classification_report(test_y.values, test_y_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2285
           1       1.00      1.00      1.00       433

    accuracy                           1.00      2718
   macro avg       1.00      1.00      1.00      2718
weighted avg       1.00      1.00      1.00      2718

